# Test for 4 classes

In [2]:
# Imports

#General
import numpy as np
import pickle
import itertools
import matplotlib.pyplot as plt

# System
import os, fnmatch

# Random Seed
from numpy.random import seed
seed(1)

# Machine Learning
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import recall_score, precision_score, accuracy_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.svm import LinearSVC, SVC

# Audio
import librosa.display, librosa

# Configurations
path='./data/instruments/'
nClasses = 4

In [3]:
# Get files in data path for wav

files = []
for root, dirnames, filenames in os.walk(path):
    for filename in fnmatch.filter(filenames, '*.wav'):
        files.append(os.path.join(root, filename))

print("found %d audio files in %s"%(len(files),path))
#print("files=" + str(files))

found 644 audio files in ./data/instruments/


In [6]:
# Get Labels
labels =[]
classes=['bassoon','clarinet','saxphone', 'violin']
color_dict={'bassoon':'blue', 'clarinet':'red', 'saxphone':'green', 'violin':'black'}
color_list=[]
for filename in files:
    for name in classes:
        if fnmatch.fnmatchcase(filename, '*'+name+'*'):
            labels.append(name)
            color_list.append(color_dict[name])
            break
    else:
        labels.append('other')

print("labels.len=" + str(len(labels)))
print("color_list.len=" + str(len(color_list)))
#print("labels=" + str(labels))
#print("color_list=" + str(color_list))

labels.len=644
color_list.len=644


In [7]:
# Encode Labels
labelencoder = LabelEncoder()
labelencoder.fit(labels)
print(len(labelencoder.classes_), "classes:", ", ".join(list(labelencoder.classes_)))
classes_num = labelencoder.transform(labels)
print("classes_num.shape=" + str(classes_num.shape))
#print("classes_num=" + str(classes_num))

4 classes: bassoon, clarinet, saxphone, violin
classes_num.shape=(644,)


In [13]:
# Parameters
# Signal Processing Parameters
fs = 44100         # Sampling Frequency
n_fft = 2048       # length of the FFT window
hop_length = 512   # Number of samples between successive frames
n_mels = 128     # Number of Mel bands
n_mfcc = 15        # Number of MFCCs

# Machine Learning Parameters
testset_size = 0.25 #Percentage of data for Testing

In [9]:
# Define Function to Calculate MFCC, Delta_MFCC and Delta2_MFCC
def get_features(y, sr=fs):
    S = librosa.feature.melspectrogram(y, sr=fs, n_mels=n_mels)
    mfcc = librosa.feature.mfcc(S=librosa.power_to_db(S), n_mfcc=n_mfcc)
    feature_vector = np.mean(mfcc,1)
    #feature_vector = (feature_vector-np.mean(feature_vector))/np.std(feature_vector)
    return feature_vector

In [10]:
# Load audio files, calculate features and create feature vectors
feature_vectors = []
sound_paths = []
for i,f in enumerate(files):
    #print ("get %d of %d = %s"%(i+1, len(files), f))
    try:
        y, sr = librosa.load(f, sr=fs)
        y/=y.max() #Normalize
        if len(y) < 2:
            print("Error loading %s" % f)
            continue
        feat = get_features(y, sr)
        #print("feat.shape=" + str(feat.shape))
        #feat.shape=(13,)
        feature_vectors.append(feat)
        sound_paths.append(f)
    except Exception as e:
        print("Error loading %s. Error: %s" % (f,e))
        
print("Calculated %d feature vectors"%len(feature_vectors))
#print("feature_vectors.type=" + str(type(feature_vectors)))
#print("feature_vectors[0].type=" + str(type(feature_vectors[0])))

Calculated 644 feature vectors


In [11]:
# Scale features using Standard Scaler
scaler = StandardScaler()
scaled_feature_vectors = scaler.fit_transform(np.array(feature_vectors))
print("Feature vectors shape:",scaled_feature_vectors.shape)

Feature vectors shape: (644, 15)


In [14]:
# Create Train and Test Set
splitter = StratifiedShuffleSplit(n_splits=1, test_size=testset_size, random_state=0)
splits = splitter.split(scaled_feature_vectors, classes_num)
for train_index, test_index in splits:
    train_set = scaled_feature_vectors[train_index]
    test_set = scaled_feature_vectors[test_index]
    train_classes = classes_num[train_index]
    test_classes = classes_num[test_index]

In [15]:
# Check Set Shapes
print("train_set shape:",train_set.shape)
print("test_set shape:",test_set.shape)
print("train_classes shape:",train_classes.shape)
print("test_classes shape:",test_classes.shape)

train_set shape: (483, 15)
test_set shape: (161, 15)
train_classes shape: (483,)
test_classes shape: (161,)


In [16]:
svclassifier = SVC(kernel='rbf', C = 10.0, gamma=0.1) 
svclassifier.fit(train_set, train_classes)

SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [17]:
# Predict using the Test Set
#predicted_labels = model_svm.predict(test_set)
predicted_labels = svclassifier.predict(test_set)

In [18]:
#print("predicted_labels=" + str(predicted_labels))
test_accuracy = svclassifier.score(test_set, test_classes)
print("test_accuracy=" + str(test_accuracy))

test_accuracy=1.0
